# #200 SDOF - Mass on a stick
<i>Simple column with lumped mass element at the top</i>
***

In [1]:
# Input Params
length = 5.0 #m		     Length of beam
element_length = 1

#### Connect to LUSAS and create a new model database

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
import plotly.express as px        # Plotting
from LPI import *
lusas = get_lusas_modeller()
# if lusas.existsDatabase():
#     raise Exception("This script will create a new model. Please save and close the current model and try again")

import m100_Tools_And_Helpers.Helpers as Helpers
Helpers.initialise(lusas)

In [3]:
# Create a new model
lusas.newProject("Structural", "SDOF_TEST.mdl")
# Get a reference to the model database
db = lusas.getDatabase()
# 3D model with vertical z axis
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

### Create model geometry

In [4]:
# Create a line
beam_line = Helpers.create_line([0,0,0], [0, 0, length])

### Create model attributes

In [5]:
# Create a beam mesh attribute
beam_mesh_attr = db.createMeshLine("Beam Mesh")
# Set the element type and size of elements BMI21 = 2Node thick beam elements, 1 = 1m length
beam_mesh_attr.setSize("BMI21", element_length)
# Point mass mesh attribute to represent the lumped mass
mass_mesh_attr = db.createPointMeshElementAttr("Point Mass Mesh", "PM3")

In [6]:
# Create a geometric section attribute
geometric_attr = db.createGeometricLine("Beam Geometry")
# Set the beam section properties for the section library
geometric_attr.setFromLibrary("UK Sections", "CHS (EN10210)", "508x10 CHS", 0, 0, 0)

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFGeometricLine instance at 0x2307377730960>

In [7]:
# Create a linear elastic material attribute with a mass damping ratio of 2%
material_attr = db.createIsotropicMaterial("Steel", 209_000_000, 0.3, 7.8, 10e-6, 2.0)
# Create a mass material attribute
mass_material_attr = db.create3dMassMaterial("Mass", 1.0, 1.0, 1.0, "point")

In [8]:
# Create support attribute
# Freedoms F=Free, R=Restrained, S=Spring
fixedSupport = db.createSupportStructural("Fixed")
fixedSupport.setStructural("R", "R", "R", "R", "R", "R", "F", "F", "F")

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFSupportStructural instance at 0x2307378128976>

In [9]:
gravity_load_attr = db.createLoadingBody("Gravity")
gravity_load_attr.setBody(0.0, 0.0, -9.81, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -9.81)

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFLoadingBody instance at 0x2307378267472>

### Assign attributes to model geometry

In [10]:
# Get the default assignment object
assignment = lusas.assignment().setAllDefaults()
# Assign the mesh
beam_mesh_attr.assignTo(beam_line, assignment)
# Assign the geometry
geometric_attr.assignTo(beam_line, assignment)
# Assign the material
material_attr.assignTo(beam_line, assignment)
# Assign the mass mesh to the line end point only
mass_mesh_attr.assignTo(beam_line.getEndPoint())
# Assign the mass material to the end point only
mass_material_attr.assignTo(beam_line.getEndPoint())
# Assign the fixed support to the first point
fixedSupport.assignTo(beam_line.getStartPoint(), assignment)

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFObjectSet instance at 0x2307378563552>

Create an eigenvalue frequency analysis

In [11]:
# Rename the loadcase
first_loadcase = Helpers.get_loadcase(1) # Helper method provides the correct type hint.
first_loadcase.setName("Eig Freq")
# Set the loadcase controls to be a frequency analysis searching for the minimum 5 natural frequencies
first_loadcase.setEigenvalueMaxMinControl("Frequency", "Minimum", 5)
# Set the eiegenvector normalisation procedure to mass for a frequency analysis - This just scales the resulting eigenvector and is the default setting
first_loadcase.getEigenvalueControl().setValue("NormalisationProcedure", "GlobalMass").setValue("Eigensolver", "Default")

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFControl instance at 0x2307378706256>

In [12]:
# Create a Response Spectrum Utility
attr = db.createSpectralCurve("RS1").setSpectralCurve(5.0, "acceleration/gravity", "period")
attr.addRow(0.0, 0.05333333333333333)
attr.addRow(0.02, 0.07288888888888889)
attr.addRow(0.04, 0.09244444444444444)
attr.addRow(0.06, 0.112)
attr.addRow(0.08, 0.1315555555555556)
attr.addRow(0.1, 0.1511111111111111)
attr.addRow(0.12, 0.1706666666666666)
attr.addRow(0.14, 0.1902222222222223)
attr.addRow(0.15, 0.2)
attr.addRow(0.16, 0.2)
attr.addRow(0.18, 0.2)
attr.addRow(0.2, 0.2)
attr.addRow(0.22, 0.2)
attr.addRow(0.24, 0.2)
attr.addRow(0.26, 0.2)
attr.addRow(0.28, 0.2)
attr.addRow(0.3, 0.2)
attr.addRow(0.32, 0.2)
attr.addRow(0.34, 0.2)
attr.addRow(0.36, 0.2)
attr.addRow(0.38, 0.2)
attr.addRow(0.4, 0.2)
attr.addRow(0.42, 0.1904761904761905)
attr.addRow(0.44, 0.1818181818181818)
attr.addRow(0.46, 0.1739130434782609)
attr.addRow(0.48, 0.1666666666666667)
attr.addRow(0.5, 0.16)
attr.addRow(0.52, 0.1538461538461538)
attr.addRow(0.54, 0.1481481481481481)
attr.addRow(0.56, 0.1428571428571429)
attr.addRow(0.58, 0.1379310344827586)
attr.addRow(0.6, 0.1333333333333334)
attr.addRow(0.62, 0.1290322580645161)
attr.addRow(0.64, 0.125)
attr.addRow(0.66, 0.1212121212121212)
attr.addRow(0.68, 0.1176470588235294)
attr.addRow(0.7, 0.1142857142857143)
attr.addRow(0.72, 0.1111111111111111)
attr.addRow(0.74, 0.1081081081081081)
attr.addRow(0.76, 0.1052631578947368)
attr.addRow(0.78, 0.1025641025641026)
attr.addRow(0.8, 0.1)
attr.addRow(0.82, 0.0975609756097561)
attr.addRow(0.84, 0.09523809523809525)
attr.addRow(0.86, 0.0930232558139535)
attr.addRow(0.88, 0.09090909090909093)
attr.addRow(0.9, 0.0888888888888889)
attr.addRow(0.92, 0.08695652173913043)
attr.addRow(0.94, 0.08510638297872341)
attr.addRow(0.96, 0.08333333333333334)
attr.addRow(0.98, 0.0816326530612245)
attr.addRow(1.0, 0.08)
attr.addRow(1.02, 0.0784313725490196)
attr.addRow(1.04, 0.07692307692307692)
attr.addRow(1.06, 0.07547169811320755)
attr.addRow(1.08, 0.07407407407407406)
attr.addRow(1.1, 0.07272727272727273)
attr.addRow(1.12, 0.07142857142857143)
attr.addRow(1.14, 0.07017543859649122)
attr.addRow(1.16, 0.06896551724137933)
attr.addRow(1.18, 0.06779661016949154)
attr.addRow(1.2, 0.06666666666666668)
attr.addRow(1.22, 0.06557377049180328)
attr.addRow(1.24, 0.06451612903225808)
attr.addRow(1.26, 0.0634920634920635)
attr.addRow(1.28, 0.0625)
attr.addRow(1.3, 0.06153846153846154)
attr.addRow(1.32, 0.06060606060606061)
attr.addRow(1.34, 0.05970149253731343)
attr.addRow(1.36, 0.05882352941176471)
attr.addRow(1.38, 0.05797101449275363)
attr.addRow(1.4, 0.05714285714285714)
attr.addRow(1.42, 0.05633802816901409)
attr.addRow(1.44, 0.05555555555555556)
attr.addRow(1.46, 0.05479452054794521)
attr.addRow(1.48, 0.05405405405405405)
attr.addRow(1.5, 0.05333333333333333)
attr.addRow(1.52, 0.05263157894736842)
attr.addRow(1.54, 0.05194805194805196)
attr.addRow(1.56, 0.05128205128205129)
attr.addRow(1.58, 0.05063291139240507)
attr.addRow(1.6, 0.05)
attr.addRow(1.62, 0.04938271604938271)
attr.addRow(1.64, 0.04878048780487805)
attr.addRow(1.66, 0.04819277108433735)
attr.addRow(1.68, 0.04761904761904762)
attr.addRow(1.7, 0.04705882352941177)
attr.addRow(1.72, 0.04651162790697675)
attr.addRow(1.74, 0.04597701149425288)
attr.addRow(1.76, 0.04545454545454546)
attr.addRow(1.78, 0.04494382022471911)
attr.addRow(1.8, 0.04444444444444445)
attr.addRow(1.82, 0.04395604395604396)
attr.addRow(1.84, 0.04347826086956522)
attr.addRow(1.86, 0.04301075268817205)
attr.addRow(1.88, 0.04255319148936171)
attr.addRow(1.9, 0.04210526315789474)
attr.addRow(1.92, 0.04166666666666667)
attr.addRow(1.94, 0.04123711340206186)
attr.addRow(1.96, 0.04081632653061225)
attr.addRow(1.98, 0.04040404040404041)
attr.addRow(2.0, 0.04)
attr.addRow(2.02, 0.03921184197627684)
attr.addRow(2.04, 0.03844675124951942)
attr.addRow(2.06, 0.03770383636535019)
attr.addRow(2.08, 0.03698224852071006)
attr.addRow(2.1, 0.036281179138322)
attr.addRow(2.12, 0.0355998576005696)
attr.addRow(2.14, 0.03493754913092847)
attr.addRow(2.16, 0.03429355281207133)
attr.addRow(2.18, 0.0336671997306624)
attr.addRow(2.2, 0.03305785123966942)
attr.addRow(2.22, 0.0324648973297622)
attr.addRow(2.24, 0.03188775510204082)
attr.addRow(2.26, 0.03132586733495183)
attr.addRow(2.28, 0.03077870113881194)
attr.addRow(2.3, 0.03024574669187145)
attr.addRow(2.32, 0.02972651605231867)
attr.addRow(2.34, 0.02922054204105487)
attr.addRow(2.36, 0.02872737719046252)
attr.addRow(2.38, 0.02824659275474896)
attr.addRow(2.4, 0.02777777777777778)
attr.addRow(2.42, 0.02732053821460283)
attr.addRow(2.44, 0.02687449610319807)
attr.addRow(2.46, 0.02643928878313174)
attr.addRow(2.48, 0.02601456815816858)
attr.addRow(2.5, 0.0256)
attr.addRow(2.52, 0.02519526329050139)
attr.addRow(2.54, 0.0248000496000992)
attr.addRow(2.56, 0.0244140625)
attr.addRow(2.58, 0.02403701700618953)
attr.addRow(2.6, 0.02366863905325444)
attr.addRow(2.62, 0.02330866499621234)
attr.addRow(2.64, 0.02295684113865932)
attr.addRow(2.66, 0.02261292328565775)
attr.addRow(2.68, 0.02227667631989307)
attr.addRow(2.7, 0.02194787379972565)
attr.addRow(2.72, 0.02162629757785467)
attr.addRow(2.74, 0.02131173743939475)
attr.addRow(2.76, 0.02100399075824406)
attr.addRow(2.78, 0.0207028621706951)
attr.addRow(2.8, 0.02040816326530612)
attr.addRow(2.82, 0.02011971228811428)
attr.addRow(2.84, 0.0198373338623289)
attr.addRow(2.86, 0.01956085872169788)
attr.addRow(2.88, 0.01929012345679012)
attr.addRow(2.9, 0.01902497027348395)
attr.addRow(2.92, 0.01876524676299494)
attr.addRow(2.94, 0.01851080568281735)
attr.addRow(2.96, 0.01826150474799124)
attr.addRow(2.98, 0.0180172064321427)
attr.addRow(3.0, 0.01777777777777778)
attr.addRow(3.02, 0.01754309021534143)
attr.addRow(3.04, 0.01731301939058172)
attr.addRow(3.06, 0.01708744499978641)
attr.addRow(3.08, 0.0168662506324844)
attr.addRow(3.1, 0.01664932362122789)
attr.addRow(3.12, 0.01643655489809336)
attr.addRow(3.14, 0.01622783885756015)
attr.addRow(3.16, 0.01602307322544464)
attr.addRow(3.18, 0.016)
attr.addRow(6.0, 0.016)


<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFPSD instance at 0x2307378220640>

### IMD Loadsets

In [13]:
imd = db.createIMD("IMD Support Motion Displacement X (Frequency)", 0)
imd.setSupportMotionExcitation("Displacement", "Relative", 1.0, 0.0, 0.0)
imd.setFrequencyResults(5.0, 0.0, "Amplitude")
imd.setDampingType("None")
imd.setAllModes()

imd = db.createIMD("IMD Support Motion Displacement X (Time-Step)", 0)
imd.setSupportMotionExcitation("Displacement", "Relative", 1.0, 0.0, 0.0)
imd.setTimeResults(1.0 / 5.0, "Step")
imd.setDampingType("None")
imd.setAllModes()

imd = db.createIMD("IMD Support Motion Displacement X (Time-Impulse)", 0)
imd.setSupportMotionExcitation("Displacement", "Relative", 1.0, 0.0, 0.0)
imd.setTimeResults(1.0 / 5.0, "Impulse")
imd.setDampingType("None")
imd.setAllModes()

imd = db.createIMD("IMD Support Motion Displacement X (Spectral)", 0)
imd.setSupportMotionExcitation("Displacement", "Relative", 1.0, 0.0, 0.0)
imd.setSpectralResults("CQC combination", "None")
imd.setSpectrum("RS1")
imd.setDampingType("None")
imd.setAllModes()


<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFIMD instance at 0x2307378660528>

In [14]:
# Unit acceleration load in X
acceleration_load_attr = db.createPrescribedAccelerationLoad("Unit acceleration in X").setAcceleration("U", 1.0)

In [15]:
response_time = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 1]
accelerations = [0, -1, 1.2, -0.6, 0.2, -0.3, 1, -1, 0.5, -0.5, 0, 0]
import numpy as np
frequency = 7
amplitude = 5
response_time = np.arange(0, 1.01, 0.01)
accelerations = np.sin(2* response_time * frequency * np.pi) * amplitude

if 1:
    fig = px.line(y=accelerations, x=response_time, labels={'y':"Dispacement [m]", 'x':"Response time [s]"}, 
                    markers=True, title=f"Load curve accelerations")
    fig.show()

### Transient Analysis (Time Stepping)

In [16]:
initial_time_step = 0.01 # second
total_response_time = 1  # second

In [ ]:
# Create a new analysis without an initial loadcase
transient_analysis = db.createAnalysisStructural("Transient", False)

# Add a loadcurve of non-varying loads (gravity)
load_curve_const = db.createLoadCurveTable("Gravity", 0.0, 1.0, transient_analysis.getName(), 0)
load_curve_const.setTableData([0,1], [1,1])
# Assign gravity body force
gravity_load_attr.assignToAll(lusas.assignment().setAllDefaults().setLoadset(load_curve_const))

# Add a load curve to vary the ground acceleration over time
if 1:
    # Load curve defined by acceleration data
    load_curve = db.createLoadCurveTable("Support acceleration curve", 0.0, 1.0, transient_analysis.getName(), 0)
    load_curve.setTableData(response_time, accelerations)
else:
    # Load curve defined by sinusoidal variation
    load_curve = db.createLoadCurveStandard("Support acceleration curve", "Sine", transient_analysis.getName(), 0)
    # O.5g at 7Hz, close to natural frequency of first mode
    load_curve.setStandardLoadCurve(0.0, 5, 7.0, 0.0, 1.0, "Sine", False, 0.0, 0.0) 

# Add a loadcase to the analysis for the support motion
first_loadcase = db.createLoadcase("Support acceleration", transient_analysis.getName())

# Add a transient control to the loadcase
first_loadcase.setTransientControl(0)
cntrl = first_loadcase.getTransientControl()
# Set the transient control for an implicit analysis in the time domain with an initial time step
cntrl.setTimeDomainDynamics(initial_time_step, False).setOutput().setConstants()
# The total response time
cntrl.setValue("TotalResponseTime", total_response_time)

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFControl instance at 0x2307441010128>

In [18]:
# Assign the acceleration loading to the load curve
assignment = lusas.assignment().setAllDefaults().setLoadset(load_curve)
acceleration_load_attr.assignTo(beam_line.getStartPoint(), assignment)

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFObjectSet instance at 0x2307440953280>

## Solve and open results

In [19]:
for analysis in db.getAnalyses():
    analysis.solve(True)
db.openAllResults(False)

### Set the display properties

In [20]:
lusas.view().setIsometric()
lusas.view().insertDiagramsLayer()
lusas.view().diagrams().setResultsTransformNone()
lusas.view().diagrams().setResults("Force/Moment - Thick 3D Beam", "My")


lusas.view().attributes().visualiseNone("Geometric")
lusas.view().setDeformationFactor(10.0, False, True)


In [21]:
# Helper function provides correct type hinting
def get_node(point:IFPoint) -> IFNode:
    return point.getNodes()[0]

In [22]:
import plotly.express as px

top_node = get_node(beam_line.getEndPoint())
sup_node = get_node(beam_line.getStartPoint())

response_time = []
deflections = []
reactions_fx = []
reactions_my = []
for results_loadset in first_loadcase.getResultsLoadcases():
    lusas.view().setActiveLoadset(results_loadset)

    response_time.append(results_loadset.getValue("RSPTIM"))
    
    deflections.append(top_node.getResults("Displacement", "DX")[0]) # returns tuple because of "out" (reference) arguments
    reactions_fx.append(sup_node.getResults("Reaction", "FX")[0])     # returns tuple because of "out" (reference) arguments
    reactions_my.append(sup_node.getResults("Reaction", "MY")[0])     # returns tuple because of "out" (reference) arguments

# Plot graphs
px.line(y=deflections, x=response_time, labels={'y':"Dispacement [m]", 'x':"Response time [s]"}, 
        markers=True, title=f"Max top node horizontal displacement = {max([abs(d) for d in deflections]):.3f} [m]").show()

px.line(y=reactions_fx, x=response_time, labels={'y':"Reaction X [kN]", 'x':"Response time [s]"}, 
        markers=True, title=f"Max support node horizontal reaction = {max([abs(r) for r in reactions_fx]):.2f} [kN]").show()

px.line(y=reactions_my, x=response_time, labels={'y':"Reaction My [kN.m]", 'x':"Response time [s]"}, 
        markers=True, title=f"Max support node moment reaction = {max([abs(r) for r in reactions_my]):.2f} [kN.m]").show()